### Pinecone Vector DB



In [2]:
## Create your index and apikey from here https://www.pinecone.io/
api_key="provide your apikey"

In [1]:
!pip install -qU langchain langchain-pinecone langchain-openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires rich<14,>=10.14.0, but you have rich 14.1.0 which is incompatible.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\anems\anaconda3\python.exe -m pip install --upgrade pip


In [3]:
from pinecone import Pinecone
pc=Pinecone(api_key=api_key)

In [4]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small",dimensions=1024,api_key="openai_api_key")

In [5]:
from pinecone import ServerlessSpec

index_name = "rag"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '74d5cd5b1411fca47c02a534e59d8a4b', 'date': 'Sat, 23 Aug 2025 09:04:19 GMT', 'content-type': 'text/html', 'server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [6]:
index

In [7]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [8]:
vector_store

In [9]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [10]:
vector_store.add_documents(documents=documents)

['319fd410-cf6f-4e54-a1ad-56c3573943e9',
 'da12c62d-6b7e-45dc-834a-c0e7c145f606',
 'c4e76fea-0f15-4404-a8db-f83462bb7553',
 '91f15ee8-9617-4c16-8fa3-65573c97093f',
 'c2b0f737-3ade-4d32-ae95-0656782cac6f',
 '05e6dcfc-6e9f-45c6-9147-421ce1d7a7ef',
 'd542df32-3a84-4fe6-95c6-5ad00fca866f',
 '5108e39c-5d2b-4675-875a-adeed65518e2',
 'ab3af2be-29ed-451f-87d1-8b18ad388e7c',
 '75f3a6bf-a79f-4a60-a976-ecc14e1db909']

In [11]:
### Query Directly
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [12]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.572665] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [13]:
### Retriever
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.4},
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(id='91f15ee8-9617-4c16-8fa3-65573c97093f', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]